# Notebook d'exemple : simulation d'un modèle de petite dimension

Ce notebook (et les données d'exemple) accompagne la librairie de modélisation macroéconomique  https://github.com/InseeFrLab/Macronometrics . 

Pour l'installer :
* ```git clone https://github.com/InseeFrLab/Macronometrics.git ```
* ```python setup.py install ```
* vérifier le répertoire /donnees/colibri : les fichiers ```small_db.csv``` et ```small_coeffs.csv``` doivent être présents.
* vérifier le répertoire /codes_modeles : le fichier ```colibri.txt``` doit être présent.

In [ ]:
import os
import pandas as pd
import sys
import numpy as np

In [ ]:
import macronometrics.model as m

In [ ]:
import macronometrics.tools as t
import macronometrics.tools_ts as ts

# Lecture du fichier d'équations définissant le modèle #

Un objet Model est instancié à partir du fichier d'équations. Pour plus d'informations sur les variables utilisées et les équations économétriques, voir : https://www.insee.fr/fr/statistiques/1380857 (dont le modèle d'exemple est une simplification) et http://grocer.toolbox.free.fr/ pour la version Scilab.

In [ ]:
colibri=m.Model()
colibri.lexer("./codes_modeles/colibri.txt")

# Chargement du modèle #

In [ ]:
colibri

Le modèle est composé d'équations, de variables (endogènes / exogènes / policy) et de coefficients (ayant vocation à être estimés).

In [ ]:
colibri.eq_obj_dict['tc_d41_s14s3']

# Construction de la fonction de résolution #

On vérifie que le modèle n'est pas encore construit, c'est à dire que les lignes de texte qui constituent les équations du modèle n'ont pas encore été interprétées pour constituer des objets sur lesquelles on va ensuite pouvoir travailler.

In [ ]:
colibri.is_built

On lance la fonction de construction du modèle, qui va utiliser l'algorithme de décomposition pour analyser la structure du système d'équations et la réduire. Un nom de fonction doit être donné : prendre le même nom que celui du modèle.

In [ ]:
colibri.build_model("colibri")

In [ ]:
colibri.is_built

A ce stade, deux types de fichiers ont été produits (dans le répertoire ```/modeles_python/```) :
* Un fichier ```colibri.yaml``` permettant de vérifier les statuts des variables de chaque équation
* Un fichier ```colibri.py``` comprenant différentes fonctions nécessaires à la résolution du modèle

### Ouvrir le fichier colibri.yaml et comparer son contenu avec le fichier colibri.txt !

Chaque équation est désormais analysée en vue de la simulation.

In [ ]:
colibri.eq_obj_dict['tc_d41_s14s3']

# Chargement des données du modèle #

On va tout d'abord travailler avec une base où les cales ont été fixées à 0 (elles n'ont pas encore été calculées). Ces séries temporelles sont issues des comptes trimestriels publiés par l'Insee et sont rassemblées dans un fichier .csv .

In [ ]:
datamodel=pd.read_csv("./donnees/colibri/small_db.csv",index_col=0,parse_dates=True)

On vérifie l'importation sur les dernières dates. Les cales ont un suffixe "_cale" dans le nom des variables.

In [ ]:
datamodel.tail()

# Chargement des coefficients #

On construit le dictionnaire des paramètres/coefficients du modèle à partir de leur importation. Plus tard, les coefficients pourront être initialisés sans valeur et celle ci sera estimée.

In [ ]:
coeffmod = t.readcoeffs("./donnees/colibri/small_coeffs.csv")

# Calage du modèle sur données historiques (1980Q3 - 2006Q4) #

La part d'inexpliqué du modèle est quantifiée par le calcul des valeurs des cales sur une période historique. Cette étape repose sur une étape d'inversion :
 * on rend endogènes les variables de type policy (les cales) qui étaient considérées initialement exogènes
 * on rend exogènes les variables endogènes historiques
 * on simule le modèle ainsi

Nom des endogènes associées à des cales dans le modèle :

In [ ]:
endo_cales = ["tc_d11_d5","tc_d41_s14s3","tc_d4z_s14s3","tc_d5_s14e3",
              "tc_emps_d7","td_p3a_d5","td_p3g_d5","td_p3m_d1","td_p3m_d5",
              "td_p51g_d5","td_p51m_d1","td_p51m_d5","td_p51t_d1","td_p51t_d5",
              "td_p523_d1","td_p523_d3","td_p6_d1","td_p6_d5","td_p7_d1",
              "td_p7_d5","td_pint_d5"]

On va procéder à une inversion du modèle pour calculer les valeurs historiques des cales, ce qui nécessite de définir un nouveau modèle tenant compte du changement de statut des variables.

In [ ]:
colibri_inv = colibri.copy()

In [ ]:
list_policy = colibri.name_policy_list.copy()

In [ ]:
list_policy

In [ ]:
for item in list_policy :
    t.changesym(colibri_inv,"endogenous",item)

In [ ]:
for item in endo_cales :
    t.changesym(colibri_inv,"exogenous",item)

On construit le nouveau modèle :

In [ ]:
colibri_inv.build_model('colibri_inv')

Le nouveau modèle a été construit (ainsi que les fonctions de résolution associées). On procède au calage :

In [ ]:
df_calage = t.simulate(datamodel,coeffmod,"1980Q3","2006Q4",'colibri_inv')

Les cales ont été calculées.

In [ ]:
df_calage[list_policy].tail()

Si tout se passe bien, si l'on calcule à nouveau les variables endogènes du modèle en prenant comme exogènes les cales ainsi calculées, on doit retrouver exactement les valeurs historiques initiales.

In [ ]:
df_verif = t.simulate(df_calage,coeffmod,"1980Q3","2006Q4",'colibri')

In [ ]:
variation_relative=abs(100*(df_verif/datamodel-1))
stat_rel=variation_relative[colibri.name_endo_list].describe()[colibri.name_endo_list]
stat_rel

In [ ]:
variation_absolue=df_verif-datamodel
stat_abs=variation_absolue.describe()[colibri.name_endo_list]
stat_abs

# Construction d'une base de données pour tester les fonctions d'extrapolations de séries.  #

Les cales et les exogènes sont prolongées à leur dernière valeur observée. Les tendances sont prolongées affines et les dummy trimestrielles sont prolongées.

In [ ]:
#Prolongement des cales à leur dernière valeur observée
liste_policy=[]
i=0
for item in colibri.name_policy_list:
    liste_policy.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
    
db_extrapol_1=ts.extrapolate_series(df_calage,liste_policy)

In [ ]:
#Prolongement des exogènes à leur dernière valeur
liste_exo=[]
i=0
for item in colibri.name_exo_list:
    liste_exo.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
db_extrapol_1=ts.extrapolate_series(db_extrapol_1,liste_exo)

In [ ]:
#Prolongement de la tendance
db_extrapol_1=ts.extrapolate_series(db_extrapol_1,[['time','affine',['2007Q1','2010Q4'],1,1]])

In [ ]:
#Prolongement des dummy trimestrielles
liste_dummy_trim=[['trim1','dummy_trim',['2007Q1','2010Q4'],[1,0,0,0]],
                 ['trim2','dummy_trim',['2007Q1','2010Q4'],[0,1,0,0]],
                 ['trim3','dummy_trim',['2007Q1','2010Q4'],[0,0,1,0]],
                 ['trim4','dummy_trim',['2007Q1','2010Q4'],[0,0,0,1]]]

db_extrapol_1=ts.extrapolate_series(db_extrapol_1,liste_dummy_trim)

In [ ]:
db_extrapol_1[colibri.name_exo_list].tail()

Le modèle peut ensuite être simulé sur cette nouvelle base de données prolongée, pour des dates qui n'étaient pas dans la base de données historiques initiale

In [ ]:
simul_1=t.simulate(db_extrapol_1,coeffmod,'2007Q1','2010Q4','colibri')

On peut faire de même en prolongant cette fois les exogènes en taux de croissance (en prévision d'une simulation d'un compte central)

In [ ]:
#Prolongement des cales à leur dernière valeur
liste_policy=[]
i=0
for item in colibri.name_policy_list:
    liste_policy.append([item,'constant',['2007Q1','2010Q4'],'last'])
    i+=1
    
db_extrapol_2=ts.extrapolate_series(df_calage,liste_policy)

In [ ]:
#Prolongement des exogènes à un taux de croissance de 3%
liste_exo=[]
i=0
for item in colibri.name_exo_list:
    liste_exo.append([item,'taux de croissance',['2007Q1','2010Q4'],3])
    i+=1
db_extrapol_2=ts.extrapolate_series(db_extrapol_2,liste_exo)

In [ ]:
#Prolongement de la tendance
db_extrapol_2=ts.extrapolate_series(db_extrapol_2,[['time','affine',['2007Q1','2010Q4'],1,1]])

In [ ]:
#Prolongement des dummy trimestrielles
liste_dummy_trim=[['trim1','dummy_trim',['2007Q1','2010Q4'],[1,0,0,0]],
                 ['trim2','dummy_trim',['2007Q1','2010Q4'],[0,1,0,0]],
                 ['trim3','dummy_trim',['2007Q1','2010Q4'],[0,0,1,0]],
                 ['trim4','dummy_trim',['2007Q1','2010Q4'],[0,0,0,1]]]

db_extrapol_2=ts.extrapolate_series(db_extrapol_2,liste_dummy_trim)

In [ ]:
db_extrapol_2[colibri.name_exo_list].tail()

In [ ]:
simul_2=t.simulate(db_extrapol_2,coeffmod,'2007Q1','2010Q4','colibri')

# On compare les deux simulations # 
(en prévision d'une fonction permettant de présenter les résultats d'une variante)  

In [ ]:
(comp_trim_abs,comp_ann_abs)=ts.compare_series('absolu',simul_1,simul_2,colibri.name_endo_list,'2007Q1','2010Q4')

In [ ]:
comp_ann_abs

In [ ]:
(comp_trim_rel,comp_ann_rel)=ts.compare_series('relatif',simul_1,simul_2,colibri.name_endo_list,'2007Q1','2010Q4')

In [ ]:
comp_ann_rel

# Simulation d'un choc permanent de demande mondiale de 1% à partir de 1990q1 #

In [ ]:
info_dict = {"variable" : ["demmon"] , "type" : "pctge" , "value" : 1}

In [ ]:
info = [info_dict]

In [ ]:
results=t.simul_shock('colibri',coeffmod,df_calage,start_date_shock='1990Q1',end_date_shock='2006Q4',start_date_sim='1985Q1',end_date_sim='2006Q4',info_shock=info)

In [ ]:
(choc_demmon_trim,choc_demmon_an)=ts.compare_series('relatif',df_calage,results,colibri.name_endo_list,'1985Q1','2006Q4')

In [ ]:
choc_demmon_an